#### 0. LangChain에서 도구(tool) 활용 방법

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gs
24


In [2]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     #model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

/Users/eon/skRookies/myLangChain/mylangchain-app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x12ab9c4c0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12b539810> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [4]:
# @tool 데코레이터로 선언된 함수는 직접 호출할 수 없다.
#add(10,20)

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [5]:
query = '3 곱하기 5는?'
llm_result = llm.invoke(query)

print(llm_result.content)

3 곱하기 5는 **15**입니다.  

계산 과정:  
\[ 3 \times 5 = 15 \]  

도움이 필요하시면 언제든지 물어보세요! 😊


- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [6]:
llm_with_tools = llm.bind_tools([add, multiply])

print(type(llm_with_tools))
print(llm_with_tools)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x12ab9c4c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12b539810>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1') kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '숫자 a와 b를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': '숫자 a와 b를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [7]:
tool_result = llm_with_tools.invoke(query)

tool_result

AIMessage(content='[질문 "3 곱하기 5는?"에 답하기 위해 곱셈 연산이 필수적이며, 제공된 함수 중 \'multiply\'만이 이 연산을 직접적으로 수행할 수 있기 때문에 해당 함수를 호출합니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-de4a0881dcbc459a81c5330d238d2dd7', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 578, 'total_tokens': 633, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'ff664ed3-7c69-4d30-8bf9-1901c510e425', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--03b66d6d-6aeb-4cb8-8a3e-1c6b96a98de5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'chatcmpl-tool-de4a0881dcbc459a81c5330d238d2dd7', 'type': 'tool_call'}], u

In [8]:
tool_result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'chatcmpl-tool-de4a0881dcbc459a81c5330d238d2dd7',
  'type': 'tool_call'}]

In [9]:
#query = "안녕하세요? 오늘의 날씨는 어떤가요?"
query = "10 더하기 5는 얼마인가요?"
tool_result = llm_with_tools.invoke(query)
print(tool_result.tool_calls)

[{'name': 'add', 'args': {'a': 10, 'b': 5}, 'id': 'chatcmpl-tool-a311d4b5c49e4fceb438decdd200d318', 'type': 'tool_call'}]


In [10]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

query = "20과 10을 곱한 결과는?"
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 

print(message_list)


[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={})]


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억하기

In [11]:
# llm_with_tools 는 LLM과 Tool이 연동된 RunnableBinding 객체
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content="[질문에서 20과 10을 곱한 결과를 요청했기 때문에 'multiply' 함수가 필수적으로 필요합니다. 다른 방법으로는 일반 지식으로 답할 수 있지만, 주어진 규칙에 따라 함수 사용이 요구될 때 명시적으로 호출해야 합니다.] \n\n정답: 200", additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 582, 'total_tokens': 656, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '62f876b6-6f7c-4f35-81ad-91395126df76', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--03fd0743-5b11-42be-a726-c4590dde31b9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20, 'b': 10}, 'id': 'chatcmpl-tool-f600cd41d756417f882

In [12]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b',
  'type': 'tool_call'}]

In [13]:
from pprint import pprint

message_list.append(ai_message)

pprint(message_list)

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="[질문에서 20과 10을 곱한 결과를 요청했기 때문에 'multiply' 함수가 필수적으로 필요합니다. 다른 방법으로는 일반 지식으로 답할 수 있지만, 주어진 규칙에 따라 함수 사용이 요구될 때 명시적으로 호출해야 합니다.] \n\n정답: 200", additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 582, 'total_tokens': 656, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '62f876b6-6f7c-4f35-81ad-91395126df76', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--03fd0743-5b11-42be-a726-c4590dde31b9-0', tool_calls=[{'

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [14]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b',
  'type': 'tool_call'}]

In [15]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

print(tool_message)

content='200' name='multiply' tool_call_id='chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b'


- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [16]:
message_list.append(tool_message)

pprint(message_list)

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="[질문에서 20과 10을 곱한 결과를 요청했기 때문에 'multiply' 함수가 필수적으로 필요합니다. 다른 방법으로는 일반 지식으로 답할 수 있지만, 주어진 규칙에 따라 함수 사용이 요구될 때 명시적으로 호출해야 합니다.] \n\n정답: 200", additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 582, 'total_tokens': 656, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '62f876b6-6f7c-4f35-81ad-91395126df76', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--03fd0743-5b11-42be-a726-c4590dde31b9-0', tool_calls=[{'

In [17]:
tool_result = llm_with_tools.invoke(message_list)

pprint(tool_result)

AIMessage(content='20과 10을 곱한 결과는 **200**입니다. \n\n계산 과정:  \n`20 × 10 = 200`  \n\n함수 호출 결과:\n\n→ `200`  \n\n따라서 최종 답은 **200**입니다.', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-6cce401bd0094da38bd01f46e9d54f6f', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 672, 'total_tokens': 752, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '0c94387c-aa69-4858-aba3-af19dfc5ed6d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--125cfef4-142b-47cb-b03f-b12c479e965d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20, 'b': 10}, 'id': 'chatcmpl-tool-6cce401bd0094da38bd01f46e9d54f6f', 'type': 'too

- `message_list`의 순서를 기억하기

In [18]:
message_list

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="[질문에서 20과 10을 곱한 결과를 요청했기 때문에 'multiply' 함수가 필수적으로 필요합니다. 다른 방법으로는 일반 지식으로 답할 수 있지만, 주어진 규칙에 따라 함수 사용이 요구될 때 명시적으로 호출해야 합니다.] \n\n정답: 200", additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f600cd41d756417f8821dba5c3dedc7b', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 582, 'total_tokens': 656, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '62f876b6-6f7c-4f35-81ad-91395126df76', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--03fd0743-5b11-42be-a726-c4590dde31b9-0', tool_calls=[{'